## ASTR 21000, Winter 2020

## Laboratory on model parameter inference

### Distributed: Monday, March 2

import packages needed by the codes below. Run this cell first before using these codes. 

In [41]:
import numpy as np

# use jupyter "magic" command to tell it to embed plot into the notebook 
import matplotlib.pyplot as plt
%matplotlib inline

from codes.plot_utils import plot_pretty
plot_pretty(fontsize=12)

### Introduction 

This lab consists of the first exercise of next coding assignment (homework 5), which will be distributed later on Monday. 

In this homework you will redo the analysis you did in homework 3 using supernovae to constrain model parameters but with a different approach. There are two differences in the approach you will use in this exercise: 

1) you will work with the *likelihood function* instead of $\chi^2$. The likelihood function and the rationale for its use, as well as relation of $\chi^2$ to this function are described below. More precisely, you will work with the likelihood function and prior probability distributions for model parameters, which combined give the pdf called *posterior distribution function*. 

2) you will use the model that abandons assumption that supernovae type Ia are "standard candles" (i.e. have the same absolute magnitude value $M_0$). Instead, the absolute magnitude of each supernova will be corrected using two of its properties measured in observations. This requires two additional parameters, which brings the number of model parameters from 3 to 5. 

Thus, we have to map out the likelihood in 5-dimensional model parameter space, which cannot be easily done by sampling on a grid as could be done for 3D parameter space in homework 3. Thus, in the two exercises in this homework you will first find the "best fit" model parameters in the first exercise and then will learn how to estimate uncertainties by sampling 5-dimensional posterior distribution. 

### Background info: probability distribution functions (pdfs) in 1- and multiple dimensions

Recall that the 1-dimensional probability distribution function, $p(x)$, is a function of one variable $x$ and is defined as probability *density* per interval of $dx$ if it is normalized as

$$\int\limits_{-\infty}^\infty p(x)dx = 1.$$

If $x$ is measured with some units, such as $grams$ or $cm$, then $p(x)$ must have unit inverse to that of $x$, so that $p(x)dx$ is *dimensionless* (has no units) and thus can be interpreted as probability to have $x$ in the range $[x, x+dx]$. 

Some pdfs can be *conditional* in that they depend on the specific values of other parameters. For example, if I have a 2-dimensional pdf, $p(x,y)$, I can define conditional 1-dimensional pdf $p(x\vert y=2)$ which is probability distribution of $x$ under condition that $y=2$. Conditional variables have no effect on units of the conditional pdf: $p(x\vert y=2)$ has units inverse of $x$, because it is pdf of $x$ and not of $y$. 

In general, pdf can be a function of more than one parameter - $p(\mathbf{x})$ where $\mathbf{x}$ is vector of variables on which pdf depends. Everything is similar to the 1-dimensional case, except we deal with many parameters instead of one. The pdf is still probability density, but now per multi-dimensional volume element $d\mathbf{x}$ and thus has units inversely proportional to the product of units of all elements of $\mathbf{x}$. It is normalized by the integral in the multidimensional space over all dimensions: 

$$\int\limits_{-\infty}^\infty p(\mathbf{x})d\mathbf{x} = 1.$$



### Likelihood function and its relation to $\chi^2$

Suppose we have a measurement of some quantity $y_i$ with a Gaussian uncertainty $\sigma_i$. Gaussianity means that the probability distribution function for the true value of $y_{i,\rm true}$ is described by the Gaussian pdf: 

$$ p(y_{i,\rm true}) = \frac{1}{\sqrt{2\pi}\sigma}\,\exp\left[-\frac{(y_{i}-y_{i,\rm true})^2}{2\sigma_i^2}\right].$$

Likewise, if we have a model $m(\mathbf{x})$ that depends on some vector of variables $\mathbf{x}$ and that predicts the value of $y_{i, \rm model}=m(\mathbf{x})$ as a function of these parameters, then the conditional pdf of the measurement given the model parameters $\mathbf{x}$ is 

$$ p(y_{i}\vert \mathbf{x}) = \frac{1}{\sqrt{2\pi}\sigma}\,\exp\left\{-\frac{[y_{i}-y_{i, \rm model}(\mathbf{x})]^2}{2\sigma_i^2}\right\}.$$

If we integrate $p(y_{i}\vert \mathbf{x})$ over all possible values of $y_i$, we will get the probability of such measurement, under condition that model is correct. 

If we have a sequence of $N_d$ *independent* measurements $\mathbf{y}=\{y_i\}$, $i=0,\ldots, N_d-1$, we can multiply all the probabilities for each individual measurement $p(y_{i}\vert \mathbf{x})$ to get the overall probability for the measurements given the model: 

$$L(\mathbf{y}\vert\mathbf{x})\equiv p(\mathbf{y}\vert\mathbf{x}) = p(y_0\vert\mathbf{x})\,p(y_1\vert\mathbf{x})\ldots p(y_{N_d-1}\vert\mathbf{x}) = \prod\limits_{i=0}^{N_d-1} p(y_i\vert\mathbf{x}),$$
which is most often called the *likelihood function* of data given model and its parameters $\mathbf{x}$. 

Probability distributions are often strongly peaked around some value and quickly become small away from the peak. Thus, for practical reasons it is convenient to work with a logarithm of the likelihood function. 

Now if we take natural log of both sides of the equation above, log of the product of pdfs will turn into sum of logarithms. If we also use the Gaussian pdf of $p(y_{i}\vert \mathbf{x})$ we will get: 

$$\ln L(\mathbf{y}\vert\mathbf{x}) = -\frac{1}{2}\,\sum\limits_{i=0}^{N_d-1}\frac{[y_{i}-y_{i, \rm model}(\mathbf{x})]^2}{\sigma_i^2} \equiv -\frac{1}{2}\,\chi^2.$$

or $\chi^2 = -2\ln L(\mathbf{y}\vert\mathbf{x})$. Thus, we see that $\chi^2$ is directly related to the natural log of the likelihood function defined above. Minimizing $\chi^2$ as a function of $\mathbf{x}$ maximizes probability (because $\chi^2$ is $\ln L$ with the negative sign) that a sequence of measurements is consistent with the model. Thus, the values of $\mathbf{x}$ that minimize $\chi^2$ and maximize $L$ are the values for which model is most consistent with measurements or "fits" the measurements best. 

From this description it may seem that $\chi^2$ and likelihood $L$ are equivalent, so why introduce the latter? First of all, this should provide a more formal justification for using $\chi^2$ as an objective ("cost") function for minimization. 

However, there are two other important reasons for working with the likelihood rather than $\chi^2$: 

1. The above derivation shows that $\chi^2$ is formally justified when $p(y_{i}\vert \mathbf{x})$ can be assumed to have Gaussian form. This is not always the case in practice and other forms of likelihoods may need to be used, in which case there is no formal justification for $\chi^2$. 

2. Using likelihood is required if we want to impose constraints on the values of model parameters in the form of their *prior* pdfs, explained below. 

### Prior pdfs

Prior pdf encodes our prior expectation for values of a variable. For example, considering the problem of constraining values of $\Omega_{\rm m0}$ parameter using supernovae data, $\Omega_{\rm m0}$ is dimensionless mean density of matter in the universe and thus, as density, should have values $\geq 0$. Also, from other previous constraints, we know that $\Omega_{\rm m0}$ is highly unlikely to have values $\Omega_{\rm m0}>1$. These considerations can be encoded in the prior pdf for $\Omega_{\rm m0}$ taken to be uniform distribution in the range $\Omega_{\rm m0}\in[0,1]$:

$$p(\Omega_{\rm m0}) = \begin{cases} 1,\  \Omega_{\rm m0}\in[0,1]\\
0, \ \mathrm{otherwise}
\end{cases}$$

Prior pdfs for independent model parameters $x_1$, $x_2$,... can be multiplied to give total prior pdf: 

$$p(\mathbf{x}) = p(x_0)p(x_1)\ldots $$

We can multiply the likelihood by $p(\mathbf{x})$: $L(\mathbf{y}\vert\mathbf{x})\, p(\mathbf{x})$, which gives pdf of measurements given the model with parameters $\mathbf{x}$ that follow prior pdf. 

### Posterior pdf

The likelihood pdf introduced above is pdf for the measurements vector $\mathbf{y}$ under condition that model with its parameter values $\mathbf{x}$ is correct. The likelihood thus  has units inverse to that of $\mathbf{y}$. 

What if we want pdf for the model parameter values $\mathbf{x}$? By dimensional considerations, we need a pdf that has units inverse to those of $\mathbf{x}$, just like the prior pdf $p(\mathbf{x})$. 

The product of likelihood and prior introduced above, $L(\mathbf{y}\vert\mathbf{x})\, p(\mathbf{x})$, on the other hand has units of $[\mathbf{y}^{-1}\mathbf{x}^{-1}]$. If we can divide pdf of measurements *independent* of the model, $p(\mathbf{y})$, which has units $[\mathbf{y}^{-1}]$, we can divide by such pdf to get pdf that has units $[\mathbf{x}^{-1}]$:

$$\pi(\mathbf{x}\vert\mathbf{y}) = \frac{L(\mathbf{y}\vert \mathbf{x})p(\mathbf{x})}{p(\mathbf{d})}.$$

This pdf is called ***posterior pdf*** of model parameters given measurements and the above expression is called the *Bayes theorem* (which is simply reflection of the properties of the conditional probabilities and their dimensionalities).  

Note that pdf $p(\mathbf{d})$ *does not* depend on $\mathbf{x}$ and is called *evidence* in physics and astronomy literature or *marginal* in statistics literature. Because by definition of pdf, we must have $\int\pi d\mathbf{x}=1$ when this multi-dimensional integral is taken over all values of $\mathbf{x}$, $p(\mathbf{d})$ is normalization of the integral over $L(\mathbf{y}\vert \mathbf{x})p(\mathbf{x})$:

$$p(\mathbf{d})= \int L(\mathbf{y}\vert \mathbf{x})p(\mathbf{x}) d\mathbf{x}.$$

### Exercise 1: using supernova type Ia sample to measure the average density of matter and vacuum energy in our Universe: final analysis using likelihood function and priors (40 points)

The model described in <a href="hw03.ipynb"><tt>hw03.ipynb</tt></a> has 3 parameters: $\Omega_{\rm m0}$, $\Omega_\Lambda$, $\tilde{M_0}$ and predicts the difference between observed and model distance modulus to be: 

\begin{eqnarray}
\Delta\mu=\mu_{\rm obs} - \mu(z,\Omega_{\rm m0},\Omega_\Lambda) &=&m_{\rm SNIa} - M_0 - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - 5\log_{10}c + 5\log_{10}H_0 - 25\\
&=& m_{\rm SNIa} - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - \tilde{M}_0, 
\end{eqnarray}

where 

$$ \tilde{M}_0 = \left(M_0 + 5\log_{10}\,\frac{c}{H_0}+25\right).$$

This model assumes that the same absolute magnitude, $M_0$ (a measure of intrinsic supernova luminosity), is the same for all supernova. Objects for which this is true are called "standard candles" in astronomy. This is not a bad assumption. For decades astronomers did think that supernovae type Ia are standard candles. However, about 30 years ago it was shown that this is not quite true. Intrinsic absolute magnitude of supernovae of this type varies, but the variation occurs not randomly, but systematically as a function of supernova color (difference of magnitudes in different bands) and the characteristic time it takes for a supernova brightness to fade from its peak brightness. 

Both of these properties are observable and are characterized by the color parameter, $c$, related to supernova color at peak brightness and lightcurve stretch parameter, $x_1$. Astronomers found that the parameter nearly constant for all supernovae is not $M_0$ ($M_0$ varies), but the combination $M_0 - \alpha x_1 + \beta c$, where $\alpha$ and $\beta$ are parameters that are constant for all supernovae. $M_0$,  and $\alpha$ and $\beta$ are *nuisance* parameters of the model and have to be estimated from the data itself. 

Thus, we can modify our model for the distance modulus by including these two extra parameters: 

$$
\Delta\mu=\mu_{\rm obs} - \mu(z,\Omega_{\rm m0},\Omega_\Lambda)= m_{\rm SNIa} - 5\log_{10}\tilde{d}_L(z,\Omega_{\rm m0},\Omega_\Lambda) - \tilde{M}_0 + \alpha x_1 - \beta c,$$ 

with the vector of 5 parameters $\mathbf{x}=\{\Omega_{\rm m0},\Omega_\Lambda, \tilde{M}_0, \alpha,\beta\}$ and $x_1$ and $c$ stretch and color parameters measured for each supernova. 

These parameters are available in the data file that we've been using along with their observational uncertainties and can be read as shown below along with other quantities we've used previously. 

In [58]:
#zCMB, mB, emB were used before in hw 3
# x1 and ex1 are stretch parameter measured for each SN and its uncertainty
# csn and ecsn are color parameter and its uncertainty
zCMB, mB, emB, x1, ex1, csn, ecsn = np.loadtxt('data/jla_lcparams.txt', 
                                               usecols=(1, 4, 5, 6, 7, 8, 9), unpack=True)

print("read sample of %d supernovae..."%(np.size(zCMB)))

read sample of 740 supernovae...


### Observational sample of supernovae type Ia

We will use a combined sample of 740 supernovae type Ia assembled and published in a study of 
<a href="https://ui.adsabs.harvard.edu/abs/2014A%26A...568A..22B/abstract">Betoule et al. (2014)</a>. The file containing the sample is available in the course repository and was downloaded from <a href="http://supernovae.in2p3.fr/sdss_snls_jla/ReadMe.html">here</a>.

**Task 1a. (10 points)** Write a routine that computes $\ln L$ for this new 5-parameter model, where 

$$\ln L(\mathbf{y}\vert\mathbf{x}) = -\frac{1}{2}\,\sum\limits_{i=0}^{N_{\rm SN}-1}\frac{\Delta\mu^2}{\sigma_{\Delta\mu,i}^2}$$

and $\sigma_{\Delta\mu,i}^2$ is total uncertainty of the observational estimate of the distance modulus that accounts for uncertainties in $m_B$, $x_1$, and $c$, which by rules of error propagation is: 

$$\sigma_{\Delta\mu,i}^2 = \sigma_{m_B}^2 + \alpha^2\sigma_{x1}^2 + \beta^2\sigma_c^2,$$

which means that the error also depends on the two model parameters, $\alpha$ and $\beta$. 

**Task 1b. (8 points)** Test your routine by using the differential evolution routine you implemented in hw 4, or <a href="https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.differential_evolution.html"><tt>scipy.minimize.differential_evolution</tt></a> routine to find parameters that minimize $-2\ln L$ and print them out. 

To define bounds, you can use ranges for $\Omega_{\rm m0}$ and $\Omega_\Lambda$ of $[0,1]$ and for $\tilde{M}_0$ say $[20, 28]$. For $\alpha$ good choice for range is $[0.05, 0.3]$ and for $\beta$: $[1., 5.]$. Feel free to experiment with these ranges. 

The values for $\Omega_{\rm m0}$ and $\tilde{M}_0$ you should get should be quite similar (although not exactly the same) as in hw 3, but $\Omega_\Lambda$ should change quite a bit. 
Describe the changes from the values derived in hw 3 and compare the parameters you derive.

**Task 1c (2 points).** Compute the reduced $\chi^2$ for the values of the parameters that minimize the $-2\ln L$. What can you conclude about how well the model describes the supernova measurements? How does the value compares to the value obtained in hw3? 

#### Group work

If you feel that your group work in hw 3 was productive, I encourage you to continue working with your group. You can also form a new group, if you wish. In this type of exercise it is helpful to discuss approach and results in a group and such collaborative work is encouraged. 